In [ ]:
from utils import increment_path

exp_name = "concat_korquad1_datasets_10epochs"
model_output_dir = increment_path("/opt/ml/outputs/models", "/no", exp_name)
print(model_output_dir)
!python train.py --do_train\
    --run_name None\
    --output_dir $model_output_dir\
    --overwrite_output_dir False\
    --seed 42\
    --fp16 True --fp16_opt_level "O1"\
    --dataloader_pin_memory True\
    --dataloader_drop_last False\
    \
    --num_train_epochs 10.0\
    --per_device_train_batch_size 16 --gradient_accumulation_steps 4\
    --evaluation_strategy "no"\
    --save_strategy "epoch"\
    --save_total_limit 3\
    \
    --learning_rate 1e-5 --lr_scheduler_type "cosine"\
    --warmup_ratio 0.1\
    --weight_decay 0.001 --adam_beta1 0.9 --adam_beta2 0.999 --adam_epsilon 1e-8\
    --adafactor False\
    --max_grad_norm 1.0\
    \
    --group_by_length False\
    --label_smoothing_factor 0.5
    
print("end training.")

/opt/ml/outputs/models/no1_concat_korquad1_datasets_10epochs
model is from bert-base-multilingual-cased
data is from /opt/ml/input/data/train_dataset
model uses device: cuda:0
DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})
100%|███████████████████████████████████████████| 71/71 [00:40<00:00,  1.77ba/s]
{'loss': 3.7324, 'learning_rate': 3.399048266485384e-06, 'epoch': 0.34}         
{'loss': 1.0964, 'learning_rate': 6.798096532970768e-06, 'epoch': 0.68}         
{'loss': 0.7834, 'learning_rate': 9.999881607641675e-06, 'epoch': 1.02}         
{'loss': 0.6415, 'learning_rate': 9.960656738602143e-06, 'epoch': 1.36}         
 14%|████▊                             | 2075/14710 [1:30:52<9:13:25,  2.63s/it]

In [263]:
from utils import increment_path

exp_name = "mecab_with_khaiii"
test_output_dir = increment_path("/opt/ml/outputs/preds", "/no", exp_name)

dataset_path = "/opt/ml/input/data/test_dataset"
model_path = "/opt/ml/outputs/models/exp1_BM25_concat_datasets/checkpoint-1570"
!python inference.py\
    --do_predict\
    --output_dir $test_output_dir\
    --dataset_name $dataset_path\
    --model_name_or_path $model_path\
    --topk 10

data is from /opt/ml/input/data/test_dataset
model is from /opt/ml/outputs/models/exp1_BM25_concat_datasets/checkpoint-1570
model uses device: cuda:0
Dataset({
    features: ['id', 'question'],
    num_rows: 600
})
Lengths of unique contexts : 56607
Passage Embedding Loaded.
[Relevant documents exhaustive search.] done in 6.028 s
Making DataFrame dataset:: 600it [00:00, 18702.17it/s]
100%|█████████████████████████████████████████████| 6/6 [00:27<00:00,  4.61s/ba]
Init Trainer...
100%|██████████████████████████████████████| 6000/6000 [00:37<00:00, 160.25it/s]
INFO:utils_qa:Saving predictions to /opt/ml/outputs/preds/no2_mecab_with_khaiii/predictions.json.
INFO:utils_qa:Saving nbest_preds to /opt/ml/outputs/preds/no2_mecab_with_khaiii/nbest_predictions.json.
No metric can be presented because there is no correct answer given. Job done!
100%|███████████████████████████████████████| 1283/1283 [03:08<00:00,  6.80it/s]


In [216]:
eval_output_dir = increment_path("./outputs/train_dataset", "/exp", exp_name)
!python train.py\
    --do_eval\
    --output_dir $eval_output_dir\
    --model_name_or_path $model_output_dir

In [3]:
!python train.py --do_train\
    --run_name None\
    --output_dir ./models/train_dataset --overwrite_output_dir False\
    --seed 42\
    --fp16 True --fp16_opt_level "O1"\
    --dataloader_num_workers 4\
    --dataloader_pin_memory True\
    --dataloader_drop_last False\
    \
    --num_train_epochs 10.0\
    --per_device_train_batch_size 16 --gradient_accumulation_steps 1\
    --per_device_eval_batch_size 16 -- eval_accumulation_steps 1\
    --evaluation_strategy "epoch"\
    --logging_strategy "steps" --logging_steps 500\
    --save_strategy "epoch"\
    --save_steps 500 --save_total_limit 3\
    --load_best_model_at_end False\
    \
    --learning_rate 5e-5 --lr_scheduler_type "cosine"\
    --warmup_ratio 0.0 --warmup_steps 0\
    --weight_decay 0.01 --adam_beta1 0.9 --adam_beta2 0.999 --adam_epsilon 1e-8\
    --adafactor False\
    --max_grad_norm 1.0\
    \
    --group_by_length False\
    --label_smoothing_factor 0.0

Traceback (most recent call last):
  File "train.py", line 348, in <module>
    main()
  File "train.py", line 34, in main
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
  File "/opt/conda/lib/python3.7/site-packages/transformers/hf_argparser.py", line 196, in parse_args_into_dataclasses
    raise ValueError(f"Some specified arguments are not used by the HfArgumentParser: {remaining_args}")
ValueError: Some specified arguments are not used by the HfArgumentParser: ['--', 'eval_accumulation_steps', '1', '--evaluation_strategy', 'epoch', '--logging_strategy', 'steps', '--logging_steps', '500', '--save_strategy', 'epoch', '--save_steps', '500', '--save_total_limit', '3', '--load_best_model_at_end', 'False', '--learning_rate', '5e-5', '--lr_scheduler_type', 'constant_with_warmup', '--warmup_ratio', '0.0', '--warmup_steps', '0', '--weight_decay', '0', '--adam_beta1', '0.9', '--adam_beta2', '0.999', '--adam_epsilon', '1e-8', '--adafactor', 'False', '--max_gra

In [ ]:
# class EvaluationStrategy, IntervalStrategy(ExplicitEnum):
    NO = "no"
    STEPS = "steps"
    EPOCH = "epoch"

# class SchedulerType(ExplicitEnum):
    LINEAR = "linear"
    COSINE = "cosine"
    COSINE_WITH_RESTARTS = "cosine_with_restarts"
    POLYNOMIAL = "polynomial"
    CONSTANT = "constant"
    CONSTANT_WITH_WARMUP = "constant_with_warmup"

# fp16_opt_level: https://nvidia.github.io/apex/amp.html